In this notebook we summarize performance of using baseline methods as anomaly scores between events.

We consider the following settings:
- Total $p_{\rm T}$
- ${\rm MET}$
- Total multiplicity

**Note:** We do not consider baselines which use particle identity information, e.g. lepton multiplicity, since the OT methods considered do not have access to this information.

In [1]:
NEVENTS = 1000
NREPEAT = 5
NSIGFIGS = 4
MINTPR   = 0.2
BASEPATH = '/content/gdrive/My Drive/Research/AnomalyDetectionWithOT/OT_anomaly_detection/'

# Preliminaries

## Information about data

The data we are using was a part of the [ML4Jets 2021 data challenge](https://indico.cern.ch/event/980214/contributions/4413658/attachments/2278124/3870358/ml4jets_data_challenge.pdf). It is publically available in `.h5` format so it's great for testing out new methods!

**Publication:**

E. Govorkova, E. Puljak, T. Aarrestad, M. Pierini, K. A. Woźniak and J. Ngadiuba, LHC physics dataset for unsupervised New Physics detection at 40 MHz, Sci. Data 9, 118 (2022),
doi:[10.1038/s41597-022-01187-8](https://www.nature.com/articles/s41597-022-01187-8), arXiv:2107.02157

**Data:**

NOTE: The original data had several bugs in it. The links in the original materials (i.e. publication and slides) point to the incorrect Version 1 of the data. Version 2 should be used instead. [This website](https://mpp-hep.github.io/ADC2021/) contains the correct links/descriptions/usage information.

- `background_for_training.h5`: 4 million Standard Model (SM) background "training" data ([link to data](https://zenodo.org/record/5046428#.ZB9yKezMKHu))
-  `Ato4l_lepFilter_13TeV_filtered.h5`: # Neutral scalar boson events, $A \rightarrow 4l$, mass = $50$ GeV ([link to data](https://zenodo.org/record/7152590#.ZB9yROzMKHu))
- `leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5`: # Leptoquark events, ${\rm LQ} \rightarrow b \tau$ ([link to data](https://zenodo.org/record/7152599#.ZB9yZOzMKHu))
- `hToTauTau_13TeV_PU20_filtered.h5`: # Scalar boson events, $h^0 \rightarrow \tau \tau$ ([link to data](https://zenodo.org/record/7152614#.ZB9ybOzMKHt))
- `hChToTauNu_13TeV_PU20_filtered.h5`: # Charged scalar boson events, $h^\pm \rightarrow \tau \nu$ ([link to data](https://zenodo.org/record/7152617#.ZB9yf-zMKHt))
- `BlackBox_background_mix.h5`: # Mystery events ([link to data](https://zenodo.org/record/5072068#.ZB9yk-zMKHt))

In [2]:
sigAliasList    = ['sig_A', 'sig_h0', 'sig_hch', 'sig_LQ']
sigFilenameList = ['Ato4l_lepFilter_13TeV_filtered.h5', 'hToTauTau_13TeV_PU20_filtered.h5', 'hChToTauNu_13TeV_PU20_filtered.h5', 'leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5']

## Google Drive preliminaries (since we're running on Google Colab)

In [3]:
#-- "Mount" Google Drive to access data and save files/images --#
# NOTE: If running locally, comment out this cell and change the basePath accordingly
# Reference: https://stackoverflow.com/questions/49031798/when-i-use-google-colaboratory-how-to-save-image-weights-in-my-google-drive
from google.colab import drive
drive.mount('/content/gdrive')

# You will be asked to sign into a Google account and give GoogleColab access

Mounted at /content/gdrive


In [4]:
#-- Set base directory and data directory path --#
basePath    = BASEPATH
dataPath    = 'data/'
resultsPath = 'results/npz_Files/'

bkgPath    = basePath+dataPath+'background_for_training.h5'
sigPathList = []
for x in sigFilenameList:
  sigPathList.append(basePath+dataPath+x)

## Import libraries

In [5]:
import numpy as np
from numpy.random import RandomState
import numpy.ma as ma

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline

import h5py
from numpy.random import Generator, PCG64
from sklearn import metrics
import itertools

import os.path

import json

## Functions

To keep things tidy, functions are externally defined in `centralFunctions.ipynb`. We run this notebook from here which defines the contained functions as if they were written here.


In [6]:
%cd /content/gdrive/My Drive/Research/AnomalyDetectionWithOT/OT_anomaly_detection/functions/
%run centralFunctions.ipynb

/content/gdrive/My Drive/Research/AnomalyDetectionWithOT/OT_anomaly_detection/functions
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 835.4/835.4 kB 8.8 MB/s eta 0:00:00


## Load data

For more details on the data structure, see [this exploratory notebook](https://colab.research.google.com/drive/138CL8n4gCtramqaaI-i0EUZ7n4f1SWVh?usp=sharing).

In [7]:
dataDict = {}
dataDict['bkg'] = h5py.File(bkgPath, 'r')

for i in range(len(sigAliasList)):
  alias   = sigAliasList[i]
  sigPath = sigPathList[i]
  dataDict[alias] = h5py.File(sigPath, 'r')

## Sample initial data

We'll start by generating sample sets of different event classes. We'll randomly sample `2*nEvents` background events, and `nEvents` signal events (for each signal type) from the full data arrays. We fix the seeds to ensure reproducibility.

In [8]:
nEvents   = NEVENTS

In [9]:
#-- Check if file with datasamples already exists, if not sample and make one --#
sampledDataFile = basePath + dataPath + 'sampledData_nEvents%s.npz'%(str(nEvents))

if os.path.exists(sampledDataFile):
  dummyDict = np.load(sampledDataFile, allow_pickle=True)

  # Unpack contents
  scoreDict = {}
  for key in dummyDict.files:
    scoreDict[key] = dummyDict[key].item()

else:
  # Dictionary structure to keep results (accounting for repeats to get statistical uncertainty)
  scoreDict = {}
  for irepeat in range(NREPEAT):
    name_repeat = 'repeat%d'%irepeat
    scoreDict[name_repeat] = {}

  # Generate 2 different samples of nEvents Background events
  random_state            = Generator(PCG64(123))
  for irepeat in tqdm(range(NREPEAT)):
    name_repeat = 'repeat%d'%irepeat
    scoreDict[name_repeat]['bkgEvents1'] = randomDataSample(dataDict['bkg']['Particles'][:, :, 0:3], nEvents, random_state)

  random_state            = Generator(PCG64(456))
  for irepeat in tqdm(range(NREPEAT)):
    name_repeat = 'repeat%d'%irepeat
    scoreDict[name_repeat]['bkgEvents2'] = randomDataSample(dataDict['bkg']['Particles'][:, :, 0:3], nEvents, random_state)

  # For each signal type, generate a sample of nEvents Signal events
  for alias in sigAliasList:
    random_state          = Generator(PCG64(123))
    for irepeat in tqdm(range(NREPEAT)):
      name_repeat = 'repeat%d'%irepeat
      scoreDict[name_repeat][alias]      = randomDataSample(dataDict[alias]['Particles'][:, :, 0:3], nEvents, random_state)

  # Save so we don't have to do this repeatedly...
  np.savez(sampledDataFile, **scoreDict)

# Total $p_{\rm T}$

## Get ROC metrics for all repeats



In [10]:
for irepeat in range(NREPEAT):
  name_repeat = 'repeat%d'%irepeat

  #-- Put in convenient list format --#
  scoreBkg = np.sum(scoreDict[name_repeat]['bkgEvents1'][:, :, 0], axis=1)
  scoreSigList = []
  for alias in sigAliasList:
    name = alias
    scoreSigList.append(np.sum(scoreDict[name_repeat][name][:, :, 0], axis=1))

  aucList, fprList, tprList, SIList, fprInvList, F1List = calcROCmetrics(scoreBkg, scoreSigList, INTERPOLATE=True)

  #-- Store everything in a dictionary --#
  for i, alias in enumerate(sigAliasList):
    name = 'ROC_metric_'+alias
    scoreDict[name_repeat][name]           = {}
    scoreDict[name_repeat][name]['auc']    = aucList[i]
    scoreDict[name_repeat][name]['fpr']    = fprList[i]
    scoreDict[name_repeat][name]['tpr']    = tprList[i]
    scoreDict[name_repeat][name]['SI']     = SIList[i]
    scoreDict[name_repeat][name]['fprInv'] = fprInvList[i]
    scoreDict[name_repeat][name]['F1']     = F1List[i]

## Get and report average performance for tables

In [11]:
#-- Get average and std of performance metrics --#
print(scoreDict.keys())
getRepeatAvStd(scoreDict)
print(scoreDict.keys())
print(scoreDict['avStdQuantities']['sig_A'].keys())

dict_keys(['repeat0', 'repeat1', 'repeat2', 'repeat3', 'repeat4'])
Analyzing signal type = sig_A 
Analyzing signal type = sig_h0 
Analyzing signal type = sig_hch 
Analyzing signal type = sig_LQ 
dict_keys(['repeat0', 'repeat1', 'repeat2', 'repeat3', 'repeat4', 'avStdQuantities'])
dict_keys(['auc', 'fpr', 'SI', 'fprInv', 'F1'])


In [12]:
#-- Report results for tables--#
indx = indxOfCertainTPR([np.linspace(0, 1, 101)], TPRval = 0.3)[0] # Assuming base TPR value

print("AUC:")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['auc']['mean'], scoreDict['avStdQuantities'][alias]['auc']['std'])

print("Inverse FPR at TPR=0.3")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['fprInv']['mean'][indx], scoreDict['avStdQuantities'][alias]['fprInv']['std'][indx])

print("SI at TPR=0.3")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['SI']['mean'][indx], scoreDict['avStdQuantities'][alias]['SI']['std'][indx])

calcMaxAvF1(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=10)

calcMaxAvSI(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=10)

AUC:
    sig_A, mean, std:  0.7408638 0.01102442360216625
    sig_h0, mean, std:  0.7076381999999999 0.008914757055579208
    sig_hch, mean, std:  0.9257288000000001 0.003987905886552464
    sig_LQ, mean, std:  0.8421214000000001 0.006928748331408755
Inverse FPR at TPR=0.3
    sig_A, mean, std:  13.272357723577235 1.4677074077689922
    sig_h0, mean, std:  19.97127919708565 3.7379124703238484
    sig_hch, mean, std:  88.2886557886558 17.08284449813864
    sig_LQ, mean, std:  19.85070563793968 1.315182258236403
SI at TPR=0.3
    sig_A, mean, std:  1.09060547696791 0.05907015113311753
    sig_h0, mean, std:  1.333905158068291 0.12001957045851167
    sig_hch, mean, std:  2.7925766913507273 0.2745823451662073
    sig_LQ, mean, std:  1.3345597860069636 0.04425811968743019
Max F1 (TPR):
    sig_A is 0.714359 $\pm$ 0.00319996 (TPR = 0.86)
    sig_h0 is 0.671479 $\pm$ 0.00837396 (TPR = 0.77)
    sig_hch is 0.863931 $\pm$ 0.00426106 (TPR = 0.89)
    sig_LQ is 0.793313 $\pm$ 0.00799211 (TPR = 0.

In [13]:
# Same order as above but easier for copying over to draft
print("AUC:")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['auc']['mean'], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['auc']['std'], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

print("Inverse FPR at TPR=0.3")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['fprInv']['mean'][indx], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['fprInv']['std'][indx], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

print("SI at TPR=0.3")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['SI']['mean'][indx], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['SI']['std'][indx], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

calcMaxAvF1(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=NSIGFIGS, SIMPLE=True)

calcMaxAvSI(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=NSIGFIGS, SIMPLE=True)

AUC:
    0.7409  $\pm$  0.01102
    0.7076  $\pm$  0.008915
    0.9257  $\pm$  0.003988
    0.8421  $\pm$  0.006929
Inverse FPR at TPR=0.3
    13.27  $\pm$  1.468
    19.97  $\pm$  3.738
    88.29  $\pm$  17.08
    19.85  $\pm$  1.315
SI at TPR=0.3
    1.091  $\pm$  0.05907
    1.334  $\pm$  0.12
    2.793  $\pm$  0.2746
    1.335  $\pm$  0.04426
Max F1 (TPR):
    0.7144 $\pm$ 0.0032 (0.86)
    0.6715 $\pm$ 0.008374 (0.77)
    0.8639 $\pm$ 0.004261 (0.89)
    0.7933 $\pm$ 0.007992 (0.9)
Max SI (TPR):
    1.2 $\pm$ 0.04191 (0.68)
    1.396 $\pm$ 0.2258 (0.2)
    3.13 $\pm$ 0.6426 (0.2)
    1.611 $\pm$ 0.06085 (0.65)


# ${\rm MET}$

## Get ROC metrics for all repeats



In [14]:
for irepeat in range(NREPEAT):
  name_repeat = 'repeat%d'%irepeat

  #-- Put in convenient list format --#
  scoreBkg = scoreDict[name_repeat]['bkgEvents1'][:, 0, 0]
  scoreSigList = []
  for alias in sigAliasList:
    name = alias
    scoreSigList.append(scoreDict[name_repeat][name][:, 0, 0])

  aucList, fprList, tprList, SIList, fprInvList, F1List = calcROCmetrics(scoreBkg, scoreSigList, INTERPOLATE=True)

  #-- Store everything in a dictionary --#
  for i, alias in enumerate(sigAliasList):
    name = 'ROC_metric_'+alias
    scoreDict[name_repeat][name]           = {}
    scoreDict[name_repeat][name]['auc']    = aucList[i]
    scoreDict[name_repeat][name]['fpr']    = fprList[i]
    scoreDict[name_repeat][name]['tpr']    = tprList[i]
    scoreDict[name_repeat][name]['SI']     = SIList[i]
    scoreDict[name_repeat][name]['fprInv'] = fprInvList[i]
    scoreDict[name_repeat][name]['F1']     = F1List[i]

## Get and report average performance for tables

In [15]:
#-- Get average and std of performance metrics --#
print(scoreDict.keys())
getRepeatAvStd(scoreDict)
print(scoreDict.keys())
print(scoreDict['avStdQuantities']['sig_A'].keys())

dict_keys(['repeat0', 'repeat1', 'repeat2', 'repeat3', 'repeat4', 'avStdQuantities'])
Analyzing signal type = sig_A 
Analyzing signal type = sig_h0 
Analyzing signal type = sig_hch 
Analyzing signal type = sig_LQ 
dict_keys(['repeat0', 'repeat1', 'repeat2', 'repeat3', 'repeat4', 'avStdQuantities'])
dict_keys(['auc', 'fpr', 'SI', 'fprInv', 'F1'])


In [16]:
#-- Report results for tables--#
indx = indxOfCertainTPR([np.linspace(0, 1, 101)], TPRval = 0.3)[0] # Assuming base TPR value

print("AUC:")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['auc']['mean'], scoreDict['avStdQuantities'][alias]['auc']['std'])

print("Inverse FPR at TPR=0.3")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['fprInv']['mean'][indx], scoreDict['avStdQuantities'][alias]['fprInv']['std'][indx])

print("SI at TPR=0.3")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['SI']['mean'][indx], scoreDict['avStdQuantities'][alias]['SI']['std'][indx])

calcMaxAvF1(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=10)

calcMaxAvSI(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=10)

AUC:
    sig_A, mean, std:  0.36896680000000004 0.013911965172469369
    sig_h0, mean, std:  0.5977423 0.014637859042906515
    sig_hch, mean, std:  0.837106 0.009308873895375365
    sig_LQ, mean, std:  0.5609084 0.010769179404207165
Inverse FPR at TPR=0.3
    sig_A, mean, std:  2.013836534749597 0.10816006365052688
    sig_h0, mean, std:  8.261535493504272 1.0821818554882103
    sig_hch, mean, std:  388.8888888888889 335.18006953329683
    sig_LQ, mean, std:  4.779528052418213 0.3807211172175657
SI at TPR=0.3
    sig_A, mean, std:  0.42553268965019014 0.011428930548330675
    sig_h0, mean, std:  0.8601452205789375 0.05535441395153976
    sig_hch, mean, std:  5.283788870110437 2.2127587833847278
    sig_LQ, mean, std:  0.6551783608881712 0.026306666045254737
Max F1 (TPR):
    sig_A is 0.666667 $\pm$ 0 (TPR = 1)
    sig_h0 is 0.666667 $\pm$ 0 (TPR = 1)
    sig_hch is 0.754188 $\pm$ 0.00731997 (TPR = 0.76)
    sig_LQ is 0.666667 $\pm$ 0 (TPR = 1)
Max SI (TPR):
    sig_A is 0.99995 $\pm$ 

In [17]:
# Same order as above but easier for copying over to draft
print("AUC:")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['auc']['mean'], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['auc']['std'], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

print("Inverse FPR at TPR=0.3")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['fprInv']['mean'][indx], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['fprInv']['std'][indx], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

print("SI at TPR=0.3")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['SI']['mean'][indx], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['SI']['std'][indx], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

calcMaxAvF1(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=NSIGFIGS, SIMPLE=True)

calcMaxAvSI(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=NSIGFIGS, SIMPLE=True)

AUC:
    0.369  $\pm$  0.01391
    0.5977  $\pm$  0.01464
    0.8371  $\pm$  0.009309
    0.5609  $\pm$  0.01077
Inverse FPR at TPR=0.3
    2.014  $\pm$  0.1082
    8.262  $\pm$  1.082
    388.9  $\pm$  335.2
    4.78  $\pm$  0.3807
SI at TPR=0.3
    0.4255  $\pm$  0.01143
    0.8601  $\pm$  0.05535
    5.284  $\pm$  2.213
    0.6552  $\pm$  0.02631
Max F1 (TPR):
    0.6667 $\pm$ 0 (1)
    0.6667 $\pm$ 0 (1)
    0.7542 $\pm$ 0.00732 (0.76)
    0.6667 $\pm$ 0 (1)
Max SI (TPR):
    1 $\pm$ 1.11e-16 (1)
    1.049 $\pm$ 0.1412 (0.2)
    8.164 $\pm$ 7.003 (0.22)
    1 $\pm$ 1.11e-16 (1)


# Total multiplicity

## Get ROC metrics for all repeats



In [18]:
for irepeat in range(NREPEAT):
  name_repeat = 'repeat%d'%irepeat

  #-- Put in convenient list format --#
  scoreBkg = np.count_nonzero(scoreDict[name_repeat]['bkgEvents1'][:, :, 0], axis=1)
  scoreSigList = []
  for alias in sigAliasList:
    name = alias
    scoreSigList.append(np.count_nonzero(scoreDict[name_repeat][name][:, :, 0], axis=1))

  aucList, fprList, tprList, SIList, fprInvList, F1List = calcROCmetrics(scoreBkg, scoreSigList, INTERPOLATE=True)

  #-- Store everything in a dictionary --#
  for i, alias in enumerate(sigAliasList):
    name = 'ROC_metric_'+alias
    scoreDict[name_repeat][name]           = {}
    scoreDict[name_repeat][name]['auc']    = aucList[i]
    scoreDict[name_repeat][name]['fpr']    = fprList[i]
    scoreDict[name_repeat][name]['tpr']    = tprList[i]
    scoreDict[name_repeat][name]['SI']     = SIList[i]
    scoreDict[name_repeat][name]['fprInv'] = fprInvList[i]
    scoreDict[name_repeat][name]['F1']     = F1List[i]

## Get and report average performance for tables

In [19]:
#-- Get average and std of performance metrics --#
print(scoreDict.keys())
getRepeatAvStd(scoreDict)
print(scoreDict.keys())
print(scoreDict['avStdQuantities']['sig_A'].keys())

dict_keys(['repeat0', 'repeat1', 'repeat2', 'repeat3', 'repeat4', 'avStdQuantities'])
Analyzing signal type = sig_A 
Analyzing signal type = sig_h0 
Analyzing signal type = sig_hch 
Analyzing signal type = sig_LQ 
dict_keys(['repeat0', 'repeat1', 'repeat2', 'repeat3', 'repeat4', 'avStdQuantities'])
dict_keys(['auc', 'fpr', 'SI', 'fprInv', 'F1'])


In [20]:
#-- Report results for tables--#
indx = indxOfCertainTPR([np.linspace(0, 1, 101)], TPRval = 0.3)[0] # Assuming base TPR value

print("AUC:")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['auc']['mean'], scoreDict['avStdQuantities'][alias]['auc']['std'])

print("Inverse FPR at TPR=0.3")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['fprInv']['mean'][indx], scoreDict['avStdQuantities'][alias]['fprInv']['std'][indx])

print("SI at TPR=0.3")
for alias in sigAliasList:
  print("    %s, mean, std: "%alias, scoreDict['avStdQuantities'][alias]['SI']['mean'][indx], scoreDict['avStdQuantities'][alias]['SI']['std'][indx])

calcMaxAvF1(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=10)

calcMaxAvSI(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=10)

AUC:
    sig_A, mean, std:  0.8913582 0.0060063902853544114
    sig_h0, mean, std:  0.7376579 0.009054392521864718
    sig_hch, mean, std:  0.8782789 0.0053938982693410645
    sig_LQ, mean, std:  0.8750064999999999 0.0067836054277942755
Inverse FPR at TPR=0.3
    sig_A, mean, std:  82.14749704479041 17.130084935370306
    sig_h0, mean, std:  13.562295807510804 1.3655712988369118
    sig_hch, mean, std:  46.852930517535945 5.802776053871595
    sig_LQ, mean, std:  38.147074291748574 5.1720390239926575
SI at TPR=0.3
    sig_A, mean, std:  2.694239207417778 0.2674669588806679
    sig_h0, mean, std:  1.1025591623167532 0.057445781482230514
    sig_hch, mean, std:  2.04481907279447 0.12456058232046116
    sig_LQ, mean, std:  1.8454160978143737 0.1199634803923265
Max F1 (TPR):
    sig_A is 0.824899 $\pm$ 0.00731606 (TPR = 0.83)
    sig_h0 is 0.721246 $\pm$ 0.00291806 (TPR = 0.91)
    sig_hch is 0.806363 $\pm$ 0.00650824 (TPR = 0.91)
    sig_LQ is 0.811714 $\pm$ 0.00823247 (TPR = 0.93)
Max SI

In [21]:
# Same order as above but easier for copying over to draft
print("AUC:")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['auc']['mean'], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['auc']['std'], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

print("Inverse FPR at TPR=0.3")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['fprInv']['mean'][indx], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['fprInv']['std'][indx], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

print("SI at TPR=0.3")
for alias in sigAliasList:
  mean = roundToSigFig(scoreDict['avStdQuantities'][alias]['SI']['mean'][indx], NSIGFIGS)
  std  = roundToSigFig(scoreDict['avStdQuantities'][alias]['SI']['std'][indx], NSIGFIGS)
  print('   ', mean, ' $\pm$ ', std )

calcMaxAvF1(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=NSIGFIGS, SIMPLE=True)

calcMaxAvSI(scoreDict, sigAliasList=sigAliasList, minTPR=MINTPR, NSIGFIGS=NSIGFIGS, SIMPLE=True)

AUC:
    0.8914  $\pm$  0.006006
    0.7377  $\pm$  0.009054
    0.8783  $\pm$  0.005394
    0.875  $\pm$  0.006784
Inverse FPR at TPR=0.3
    82.15  $\pm$  17.13
    13.56  $\pm$  1.366
    46.85  $\pm$  5.803
    38.15  $\pm$  5.172
SI at TPR=0.3
    2.694  $\pm$  0.2675
    1.103  $\pm$  0.05745
    2.045  $\pm$  0.1246
    1.845  $\pm$  0.12
Max F1 (TPR):
    0.8249 $\pm$ 0.007316 (0.83)
    0.7212 $\pm$ 0.002918 (0.91)
    0.8064 $\pm$ 0.006508 (0.91)
    0.8117 $\pm$ 0.008232 (0.93)
Max SI (TPR):
    2.762 $\pm$ 0.1713 (0.42)
    1.167 $\pm$ 0.02742 (0.72)
    2.092 $\pm$ 0.09866 (0.54)
    1.946 $\pm$ 0.07213 (0.51)
